In [1]:
from astropy.table import *
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
from astropy.table import hstack
import os
import seaborn as sns
from scipy.stats import binned_statistic
import matplotlib
import matplotlib.backends.backend_pdf
from scipy.stats import chi2, binned_statistic
from cosmic_calc import * 
from scipy.optimize import curve_fit
from bisect import bisect
from datetime import datetime
from scipy.optimize import curve_fit
from statsmodels.stats.weightstats import DescrStatsW

plt.rc('font', family='serif'), plt.rc('xtick', labelsize=18), plt.rc('ytick', labelsize=18)
plt.rcParams['savefig.dpi'] = 300
plt.rc('text',usetex=True)
matplotlib.rcParams['xtick.direction'] = 'in'
matplotlib.rcParams['ytick.direction'] = 'in'
np.set_printoptions(precision=3)

In [2]:
dev_path = '/Users/lejay/research/lephare_dev/my_code/'
output_dir = dev_path+'output_cats/'
graham_output_dir = dev_path + 'graham_output_cats_new/'

#### Trim some columns

In [4]:
# trim catalog for 6B redshift derived physical parameters

prefix = 'pcat_z6B_' # '','v9pcat'

cat_names = ['COSMOS_deep','XMM-LSS_deep']
# cat_names = ['XMM-LSS_deep']
for cat_name in cat_names:
    print(graham_output_dir+prefix+cat_name+'_cat_out_nomaglimit_nonir_il_nz_prior_formass.fits')
    if cat_name != 'XMM-LSS_deep':
        output_cat = Table.read(graham_output_dir+prefix+cat_name+'_cat_out_nomaglimit_nonir_il_nz_prior_formass.fits')
    else:
        output_cat = Table.read(graham_output_dir+prefix+cat_name+'_cat_out_nomaglimit_noirac_il_nz_prior_formass.fits')
    
    keys_set1 = [key for key in output_cat.keys() if 'NBAND_USED' in key]
    keys_set2 = [key for key in output_cat.keys() if 'CONTEXT' in key]
    keys_set3 = [key for key in output_cat.keys() if 'Z_BEST' in key]
    keys_set4 = [key for key in output_cat.keys() if 'Z_ML' in key]
    keys_other = ['IDENT','CONTEXT','NBAND_USED','EBV_BEST', 'STRING_INPUT']
    if cat_name == 'COSMOS_deep':
        keys_other.append('HSC_i_MAGERR_AUTO')
    else:
        keys_other.append('FLAG_FIELD')
    keys = keys_set1 + keys_set2 + keys_set3 + keys_set4+ keys_other
    
    for key in keys:
        try:
            output_cat.remove_column(key)
        except KeyError:
            print(key, 'already removed')
        
    print(len(output_cat.keys()))
    print('')
    
    # rename some columns
    for key in output_cat.keys():
        if '_massrun' in key:
            output_cat.rename_column(key,key.replace('_massrun',''))
    
    if cat_name != 'XMM-LSS_deep':
        output_cat.write(graham_output_dir+prefix+cat_name+'_cat_out_nomaglimit_nonir_il_nz_prior_formass.fits', overwrite=True)
    else:
        output_cat.write(graham_output_dir+prefix+cat_name+'_cat_out_nomaglimit_noirac_il_nz_prior_formass.fits', overwrite=True)
    print()

/Users/lejay/research/lephare_dev/my_code/graham_output_cats_new/pcat_z6B_COSMOS_deep_cat_out_nomaglimit_nonir_il_nz_prior_formass.fits
IDENT already removed
CONTEXT already removed
NBAND_USED already removed
EBV_BEST already removed
STRING_INPUT already removed
HSC_i_MAGERR_AUTO already removed
104


/Users/lejay/research/lephare_dev/my_code/graham_output_cats_new/pcat_z6B_XMM-LSS_deep_cat_out_nomaglimit_nonir_il_nz_prior_formass.fits
CONTEXT already removed
NBAND_USED already removed
FLAG_FIELD already removed
84




In [5]:
# the function to add photometric flags
# for a certain band, good flag requirements are detected + low uncertainty

def add_photo_flag(cat,cat_name):
    cat_matched = cat
    
    max_phot_err = 0.2
    cond_U = ( (cat_matched['u'] > 0.) & (cat_matched['u'] < 50.) & (cat_matched['u_err'] > 0.) & (cat_matched['u_err'] < max_phot_err*2) )| \
             ( (cat_matched['uS'] > 0.) & (cat_matched['uS'] < 50.) & (cat_matched['uS_err'] > 0.) & (cat_matched['uS_err'] < max_phot_err*2) )
    cond_g = ( (cat_matched['g'] > 0.) & (cat_matched['g'] < 50.) & (cat_matched['g_err'] > 0.) & (cat_matched['g_err'] < max_phot_err*2) ) 
    cond_r = ( (cat_matched['r'] > 0.) & (cat_matched['r'] < 50.) & (cat_matched['r_err'] > 0.) & (cat_matched['r_err'] < max_phot_err*2) )
    cond_i = ( (cat_matched['i'] > 0.) & (cat_matched['i'] < 50.) & (cat_matched['i_err'] > 0.) & (cat_matched['i_err'] < max_phot_err) )
    cond_z = ( (cat_matched['z'] > 0.) & (cat_matched['z'] < 50.) & (cat_matched['z_err'] > 0.) & (cat_matched['z_err'] < max_phot_err) )
    cond_y = ( (cat_matched['y'] > 0.) & (cat_matched['y'] < 50.) & (cat_matched['y_err'] > 0.) & (cat_matched['y_err'] < max_phot_err) )
    flag_optical = cond_U.astype(int) + cond_g.astype(int) + cond_r.astype(int) + cond_i.astype(int) + cond_z.astype(int) + cond_y.astype(int)
    
    if cat_name == 'COSMOS_deep' or cat_name == 'XMM-LSS_deep':
        cond_Y = ( (cat_matched['Yv'] > 0.) & (cat_matched['Yv'] < 50.) & (cat_matched['Yv_err'] > 0.) & (cat_matched['Yv_err'] < max_phot_err) )
        cond_J = ( (cat_matched['J'] > 0.) & (cat_matched['J'] < 50.) & (cat_matched['J_err'] > 0.) & (cat_matched['J_err'] < max_phot_err) )
        cond_H = ( (cat_matched['H'] > 0.) & (cat_matched['H'] < 50.) & (cat_matched['H_err'] > 0.) & (cat_matched['H_err'] < max_phot_err) )
        cond_Ks = ( (cat_matched['Ks'] > 0.) & (cat_matched['Ks'] < 50.) & (cat_matched['Ks_err'] > 0.) & (cat_matched['Ks_err'] < max_phot_err) )
        flag_nir = cond_Y.astype(int) + cond_J.astype(int) + cond_H.astype(int) + cond_Ks.astype(int)
    else:
        flag_nir = np.zeros(len(cat))
    
    if cat_name != 'XMM-LSS_deep':
        cond_ch1 = ( (cat_matched['MAG_AUTO_CH1_CORR'] > 0.) & (cat_matched['MAG_AUTO_CH1_CORR'] < 50.) & (cat_matched['MAGERR_AUTO_CH1_CORR'] < 4) )
        cond_ch2 = ( (cat_matched['MAG_AUTO_CH2_CORR'] > 0.) & (cat_matched['MAG_AUTO_CH2_CORR'] < 50.) & (cat_matched['MAGERR_AUTO_CH1_CORR'] < 4) )
        flag_irac = cond_ch1.astype(int) + cond_ch2.astype(int) 
    else:
        flag_irac = np.zeros(len(cat))
     
    flag_list = [flag_optical,flag_nir,flag_irac]
    flag_name_list = ['flag_optical','flag_nir','flag_irac']
        
    if len(cat_matched) != len(flag_optical) or len(cat_matched) != len(flag_nir) or len(cat_matched) != len(flag_irac):
        print('lengths do not match')
        return None
    else:
        col_flag_optical = Column(name='flag_optical', data=flag_optical)
        col_flag_nir = Column(name='flag_nir', data=flag_nir)
        col_flag_irac = Column(name='flag_irac', data=flag_irac)
        if 'flag_optical' in cat_matched.keys():
            cat_matched.remove_column('flag_optical')
        if 'flag_nir' in cat_matched.keys():
            cat_matched.remove_column('flag_nir')
        if 'flag_irac' in cat_matched.keys():
            cat_matched.remove_column('flag_irac')
            
        cat_matched.add_columns([col_flag_optical, col_flag_nir, col_flag_irac])
        return cat_matched

In [6]:
# 6bz based catalogs

prefix = 'pcat_z6B_'
cat_names = ['COSMOS_deep']
# cat_names = ['COSMOS_deep','XMM-LSS_deep']
for cat_name in cat_names:
    if cat_name != 'XMM-LSS_deep':
        output_cat = Table.read(graham_output_dir+prefix+cat_name+'_cat_out_nomaglimit_nonir_il_nz_prior_formass.fits')
    else:
        output_cat = Table.read(graham_output_dir+prefix+cat_name+'_cat_out_nomaglimit_noirac_il_nz_prior_formass.fits')
    
    output_cat = add_photo_flag(output_cat,cat_name)
    if cat_name != 'XMM-LSS_deep':
        output_cat.write(graham_output_dir+prefix+cat_name+'_cat_out_nomaglimit_nonir_il_nz_prior_formass.fits', overwrite=True)
    else:
        output_cat.write(graham_output_dir+prefix+cat_name+'_cat_out_nomaglimit_noirac_il_nz_prior_formass.fits', overwrite=True)

In [7]:
# color-color boundary
def boundary_basic(color1, ic_1, ic_2, slope):
    nuv_1 = ic_1
    nuv_2 = slope*color1 + ic_2
    return np.maximum(nuv_1, nuv_2)


def boundary_4d(color1, ic_1, ic_2, slope, z_factor, M_factor, z, M):
    nuv_1 = ic_1
    nuv_2 = (slope+z_factor*z+M_factor*M)*(color1 + ic_2)
    return np.maximum(nuv_1, nuv_2)


def boundary_3d(color1, ic_1, ic_2, slope, M_factor, M):
    nuv_1 = ic_1
    nuv_2 = (slope+M_factor*M)*color1 + ic_2
    return np.maximum(nuv_1, nuv_2)

In [18]:
# sfq_info in catalog
# keynames: sfq_nuvrk_myrun_[...]

prefix = 'pcat_z6B_'
sfq_fit_type = 'free'
add_sfq = 'add_sfq' # sfq_only or add_sfq
cat_type = 'all' # all or central_cosmos or xmm

##########################################
if prefix == 'pcat_' or prefix == 'pcat_z6B_':
    z_keyname = 'Z_COMBINE'
else:
    z_keyname = 'Z_ML'

detect_limit = 'nolimit'
photoz_type_show = '_z6B'
cat_names = ['COSMOS_deep','DEEP_deep','ELAIS_deep','XMM-LSS_deep']
phot_configs = ['nonir','nonir','nonir','noirac']

# phot_configs = ['nonirirac','nonir','noirac','allphot']

In [25]:
###########################################
for i,cat_name in enumerate(cat_names):
    phot_config = phot_configs[i]
    
    if cat_name == 'COSMOS_deep' or cat_name == 'XMM-LSS_deep':
        cat_full_name = graham_output_dir+prefix+cat_name+'_cat_out_nomaglimit_'+phot_config+'_il_nz_prior_formass.fits'  # all fields 
    else:
        cat_full_name = graham_output_dir+prefix.replace('z6B_','')+cat_name+'_cat_out_nomaglimit_'+phot_config+'_il_nz_prior_formass.fits'  # all fields 
        
    cat = Table.read(cat_full_name)
    sfq_types = ['fc_weighted','fm_weighted','balanced','onebin_fc','onebin_fm','onebin_bal']
    
    print(cat_full_name)
    
    #nuvrk
    diagram_type = 'NUVrK'
    print(cat_name, phot_config, sfq_fit_type)
    
    for sfq_type in sfq_types:
        print(sfq_type)
        sfq_nuvrk = []
        for i in tqdm(range(len(cat))):
            gal = cat[i]
            
            if abs(gal['MAG_ABS_r']) < 50. and abs(gal['MAG_ABS_Ks'])< 50. and abs(gal['MAG_ABS_NUV'])< 50.:
                nuvr = gal['MAG_ABS_NUV'] - gal['MAG_ABS_r']
                rk = gal['MAG_ABS_r'] - gal['MAG_ABS_Ks']
                
                if 'onebin' not in sfq_type:
                    r_factors_base = np.array([1.1,1.1,1.0,0.9])
                    if sfq_type == 'fc_weighted':
                        r_factors = r_factors_base*2.5
                    elif sfq_type == 'fm_weighted':
                        r_factors = r_factors_base/2
                    elif sfq_type == 'balanced':
                        r_factors = r_factors_base
                    else:
                        raise NameError('wrong sfq type')
                        
                    if gal[z_keyname]>0.2 and gal[z_keyname]<=0.5:
                        r = r_factors[0]
                        best_boxes_nuvrk = np.load('best_params_4d_ssfr'+photoz_type_show+'_fcfm_'+str(r)+'_0.2_0.5.npy')
                        delta_gv_nuvrk = nuvr - boundary_4d(rk,best_boxes_nuvrk[0],best_boxes_nuvrk[1],best_boxes_nuvrk[2],best_boxes_nuvrk[3],best_boxes_nuvrk[4],gal[z_keyname],gal['MASS_MED'])
                        if delta_gv_nuvrk<0:
                            sfq_nuvrk.append(1.) # sf
                        else:
                            sfq_nuvrk.append(0.) # q
                            
                    elif gal[z_keyname]>0.5 and gal[z_keyname]<=0.8:
                        r = r_factors[1]
                        best_boxes_nuvrk = np.load('best_params_4d_ssfr'+photoz_type_show+'_fcfm_'+str(r)+'_0.5_0.8.npy')
                        delta_gv_nuvrk = nuvr - boundary_4d(rk,best_boxes_nuvrk[0],best_boxes_nuvrk[1],best_boxes_nuvrk[2],best_boxes_nuvrk[3],best_boxes_nuvrk[4],gal[z_keyname],gal['MASS_MED'])
                        if delta_gv_nuvrk<0:
                            sfq_nuvrk.append(1.) # sf
                        else:
                            sfq_nuvrk.append(0.) # q
                            
                    elif gal[z_keyname]>0.8 and gal[z_keyname]<=1.1:
                        r = r_factors[2]
                        best_boxes_nuvrk = np.load('best_params_4d_ssfr'+photoz_type_show+'_fcfm_'+str(r)+'_0.8_1.1.npy')
                        delta_gv_nuvrk = nuvr - boundary_4d(rk,best_boxes_nuvrk[0],best_boxes_nuvrk[1],best_boxes_nuvrk[2],best_boxes_nuvrk[3],best_boxes_nuvrk[4],gal[z_keyname],gal['MASS_MED'])
                        if delta_gv_nuvrk<0:
                            sfq_nuvrk.append(1.) # sf
                        else:
                            sfq_nuvrk.append(0.) # q
                    elif gal[z_keyname]>1.1 and gal[z_keyname]<=1.5:
                        r = r_factors[3]
                        best_boxes_nuvrk = np.load('best_params_4d_ssfr'+photoz_type_show+'_fcfm_'+str(r)+'_1.1_1.5.npy')
                        delta_gv_nuvrk = nuvr - boundary_4d(rk,best_boxes_nuvrk[0],best_boxes_nuvrk[1],best_boxes_nuvrk[2],best_boxes_nuvrk[3],best_boxes_nuvrk[4],gal[z_keyname],gal['MASS_MED'])
                        if delta_gv_nuvrk<0:
                            sfq_nuvrk.append(1.) # sf
                        else:
                            sfq_nuvrk.append(0.) # q
                    else:
                        sfq_nuvrk.append(-99.0) 
                else:
                    if sfq_type == 'onebin_fc':
                        r = 2.5
                    elif sfq_type == 'onebin_fm':
                        r = 1.0
                    elif sfq_type == 'onebin_bal':
                        r = 0.5
                    else:
                        r = 1.0
                        
                    if gal[z_keyname]>0.1 and gal[z_keyname]<=1.1:
                        best_boxes_nuvrk = np.load('best_params_4d_ssfr'+photoz_type_show+'_fcfm_'+str(r)+'.npy') # overall sfq boundary from 0-0.9 references
                        delta_gv_nuvrk = nuvr - boundary_4d(rk,best_boxes_nuvrk[0],best_boxes_nuvrk[1],best_boxes_nuvrk[2],best_boxes_nuvrk[3],best_boxes_nuvrk[4],gal[z_keyname],gal['MASS_MED'])
                        if delta_gv_nuvrk<0:
                            sfq_nuvrk.append(1.) # sf
                        else:
                            sfq_nuvrk.append(0.) # q
                    else:
                        sfq_nuvrk.append(99.)

            else:
                sfq_nuvrk.append(99.)
            
        sfq_col_name = 'sfq_nuvrk_'+sfq_type
        sfq_col_nuvrk = Column(name=sfq_col_name, data=sfq_nuvrk)  # 1=sf, 0=q
        print(len(sfq_col_nuvrk))
        if sfq_col_name in cat.keys():
            cat.remove_column(sfq_col_name)
        cat.add_column(sfq_col_nuvrk)
        
        # high mass flag (fake massive flag)
        if 'XMM' not in cat_name:
            cond_ch1 = ( (cat['MAG_AUTO_CH1_CORR'] > 0.) & (cat['MAG_AUTO_CH1_CORR'] < 50.) & (cat['MAGERR_AUTO_CH1_CORR'] < 4.0))
            cond_ch2 = ( (cat['MAG_AUTO_CH2_CORR'] > 0.) & (cat['MAG_AUTO_CH2_CORR'] < 50.) & (cat['MAGERR_AUTO_CH1_CORR'] < 4.0))
            flag_irac = cond_ch1.astype(int) + cond_ch2.astype(int)
            
            fake_massive_cuts = np.array([9.71,9.92,10.06,10.2])
            fake_massive_flag1 = (cat['Z_COMBINE']>0.2) & (cat['Z_COMBINE']<=0.5) & (cat['MASS_MED']>fake_massive_cuts[0]) & (flag_irac==0)
            fake_massive_flag2 = (cat['Z_COMBINE']>0.5) & (cat['Z_COMBINE']<=0.8) & (cat['MASS_MED']>fake_massive_cuts[1]) & (flag_irac==0)
            fake_massive_flag3 = (cat['Z_COMBINE']>0.8) & (cat['Z_COMBINE']<=1.1) & (cat['MASS_MED']>fake_massive_cuts[2]) & (flag_irac==0)
            fake_massive_flag4 = (cat['Z_COMBINE']>1.1) & (cat['Z_COMBINE']<=1.5) & (cat['MASS_MED']>fake_massive_cuts[3]) & (flag_irac==0)
            fake_massive_flag = (fake_massive_flag1 + fake_massive_flag2 + fake_massive_flag3 + fake_massive_flag4).astype(int)
            flag_mag = (cat['i']<23) & (cat['u']<25).astype(int)
            fake_massive_flag = fake_massive_flag*flag_mag
        else:
            fake_massive_flag = np.zeros(len(cat))
        
    if add_sfq == 'sfq_only':
        cat = cat['ID','RA','DEC','Z_COMBINE','MASS_MED','i','MAG_ABS_r','MAG_ABS_Ks','MAG_ABS_NUV',sfq_col_name]
        
    #### add fake massive flag
    fake_massive_col = Column(data=fake_massive_flag, name='HIGH_MASS_FLAG')
    if 'HIGH_MASS_FLAG' in cat.keys():
        cat.remove_column('HIGH_MASS_FLAG')
    cat.add_column(fake_massive_col)
    
    ### add morphological flag psf-cmodel mag
    print(cat_name)
    print(cat.info)
    cond_morph = (cat['i_psf']-cat['i_cmodel']<-0.03*cat['i_cmodel']+22.5*0.03).astype(int)
    morph_flag_col = Column(data=cond_morph, name='i_compact_flag')
    if 'i_compact_flag' in cat.keys():
        cat.remove_column('i_compact_flag')
    cat.add_column(morph_flag_col)
    
    # write catalog
    # if add_sfq == 'sfq_only':
    #     cat.write(cat_full_name.replace('.fits','_sfq_only.fits'),overwrite=True)
    # else:
    #     cat.write(cat_full_name.replace('.fits','_sfq_added.fits'),overwrite=True)

  0%|          | 1450/1903725 [00:00<02:11, 14496.80it/s]

/Users/lejay/research/lephare_dev/my_code/graham_output_cats_new/pcat_z6B_COSMOS_deep_cat_out_nomaglimit_nonir_il_nz_prior_formass.fits
COSMOS_deep nonir free
fc_weighted


100%|██████████| 1903725/1903725 [02:01<00:00, 15644.63it/s]


1903725


  0%|          | 1444/1903725 [00:00<02:11, 14429.49it/s]

...

2596170
onebin_fm


  0%|          | 5719/2596170 [00:00<00:45, 57153.40it/s]

2596170
onebin_bal


100%|██████████| 2596170/2596170 [01:47<00:00, 24129.48it/s]


2596170
XMM-LSS_deep
<Table length=2596170>
         name          dtype  n_bad
--------------------- ------- -----
                   ID   int64     0
                   RA float64     0
                  DEC float64     0
                tract  bytes4     0
                patch  bytes3     0
              ZPHOT_G float64     0
           ZPHOT_G_6B float64     0
            isCompact    bool     0
               isStar    bool     0
        isOutsideMask   int16     0
           inside_hsc    bool     0
             inside_u    bool     0
            inside_uS    bool     0
             inside_j    bool     0
                    u float64     0
                u_err float64     0
                snr_u float64     0
                   uS float64     0
               uS_err float64     0
               snr_uS float64     1
                    g float64     0
                g_err float64     0
                snr_g float64     0
                    r float64     0
                r_er